In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import matthews_corrcoef

## Summary of Results:

$\hat Q$ is the outcome estimator, $\hat G$ is the propensity score estimator. Their respective columns tell us whether the model is misspecified $\times$ or correctly specified $\checkmark$. 

'Reduction' is the relative percent error reduction when compared against the plug-in estimator using the outcome model alone. The results are averages over 1000 simulations.

|Exp. No. | Method | $\hat Q$ | $\hat G$ | Reduction $\%$ | Rel. Error $\%$ |
| ---| --- | --- | --- | --- |--- |
| 1| TMLE | $\times$ | $\times$ | -0.128 | 1.212 |



## Problem Setup:

This example is taken from https://arxiv.org/abs/2107.00681 by Hines, Dukes, Diaz-Ordaz, and Vansteelandt (2021) and the empirical evaluation follows https://onlinelibrary.wiley.com/doi/full/10.1002/sim.7628 by Miguel Angel Luque-Fernandez, Michael Schomaker, Bernard Rachet, Mireille E. Schnitzer (2018).


The following experiments are very similar to the ones in ATE.ipynb, but this time we will fit the estimators using a neural network.

## 1. Define the DGP and some helper functions:

In [4]:

def sigm(x):
    return 1/(1 + np.exp(-x))

def inv_sigm(x):
    return np.log(x/(1-x))

def generate_data(N, seed):
    np.random.seed(seed=seed)
    z1 = np.random.binomial(1, 0.5, (N,1))
    z2 = np.random.binomial(1, 0.65, (N,1))
    z3 = np.round(np.random.uniform(0, 4, (N,1)),3)
    z4 = np.round(np.random.uniform(0, 5, (N,1)),3)
    X = np.random.binomial(1, sigm(-0.4 + 0.2*z2 + 0.15*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    Y1 = np.random.binomial(1, sigm(-1 + 1 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    Y0 = np.random.binomial(1, sigm(-1 + 0 - 0.1*z1 + 0.3*z2 + 0.25*z3 + 0.2*z4 + 0.15*z2*z4), (N,1))
    Y = Y1 * X + Y0 * (1-X)
    Z = np.concatenate([z1,z2,z3,z4],1)
    return Z, X, Y, Y1, Y0

## 2. Define the Neural Network Objects/Classes

In [5]:


def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)     

class QNet(nn.Module):
    def __init__(self, input_size, num_layers, layers_size, output_size, output_type, dropout):
        super(QNet, self).__init__()      
        
        layers = []
        layers.extend([nn.Linear(input_size, layers_size), nn.LeakyReLU()])
        for i in range(num_layers-1):
            layers.extend([nn.Linear(layers_size, layers_size), nn.LeakyReLU(), nn.Dropout(p=dropout)])
        self.net = nn.Sequential(*layers)
        
        pos_arm = []
        pos_arm.extend([nn.Linear(layers_size, layers_size), nn.LeakyReLU()])
        pos_arm.extend([nn.Linear(layers_size, output_size)])     
        
        neg_arm = []
        neg_arm.extend([nn.Linear(layers_size, layers_size), nn.LeakyReLU()])
        neg_arm.extend([nn.Linear(layers_size, output_size)])    
        
        if output_type == 'categorical':
            pos_arm.append(nn.Sigmoid())
            neg_arm.append(nn.Sigmoid())
        elif output_type == 'continuous':
            pass
        self.pos_arm = nn.Sequential(*pos_arm)
        self.neg_arm = nn.Sequential(*neg_arm)
    
        self.net.apply(init_weights) 
        self.neg_arm.apply(init_weights) 
        self.pos_arm.apply(init_weights) 


    def forward(self, X, Z):
        
        out = self.net(torch.cat([X,Z],1))
        out0 = self.neg_arm(out)
        out1 = self.pos_arm(out)
        cond = X.bool()
        return torch.where(cond, out1, out0)

    
    
class GNet(nn.Module):
    def __init__(self, input_size, num_layers, layers_size, output_size, output_type, dropout):
        super(GNet, self).__init__()      
        
        layers = []
        layers.extend([nn.Linear(input_size, layers_size), nn.LeakyReLU()])
        for i in range(num_layers-1):
            layers.extend([nn.Linear(layers_size, layers_size), nn.LeakyReLU(), nn.Dropout(p=dropout)])
        layers.extend([nn.Linear(layers_size, output_size)])

        if output_type == 'categorical':
            layers.append(nn.Sigmoid())
        elif output_type == 'continuous':
            pass
        self.net = nn.Sequential(*layers)
        self.net.apply(init_weights) 
        
    def forward(self, Z):
        return self.net(Z)

## 3. Create a Neural Network training class

In [99]:
class Trainer(object):
    def __init__(self, qnet, gnet, iterations=None, batch_size=None, test_iter=None, lr=None):
        self.qnet = qnet
        self.gnet = gnet
        self.iterations = iterations
        self.batch_size = batch_size
        self.test_iter = test_iter
        
        if lr is not None:
            self.q_optimizer = optim.Adam(qnet.parameters(), lr=lr)
            self.g_optimizer = optim.Adam(gnet.parameters(), lr=lr)
        self.bce_loss = nn.BCELoss(reduction='none')
        self.mse_loss = nn.MSELoss()
    
        
    def train(self, x, y, z):
        
        # create a small validation set
        indices = np.arange(len(x))
        np.random.shuffle(indices)
        val_inds = indices[:len(x)//8]
        train_inds = indices[len(x)//8:]
        x_val, y_val, z_val = x[val_inds], y[val_inds], z[val_inds]
        x_train, y_train, z_train = x[train_inds], y[train_inds], z[train_inds]
        
        indices = np.arange(len(x_train))
        
        train_losses_q = []
        train_losses_g = []
        test_losses_q = []
        test_losses_g = []
        
        for it in range(self.iterations):
            inds = np.random.choice(indices, self.batch_size)
            x_batch, y_batch, z_batch = x_train[inds], y_train[inds], z_train[inds]
            
            x_pred = self.gnet(z_batch)
            y_pred = self.qnet(x_batch, z_batch)
            
            q_loss = self.mse_loss(y_pred, y_batch)
                       
            weight = torch.tensor([0.7, 0.3])
            weight_ = weight[x_batch.data.view(-1).long()].view_as(x_batch)
            g_loss = (self.bce_loss(x_pred, x_batch) * weight_).mean()
            
            q_loss.backward()
            g_loss.backward()
            
            self.q_optimizer.step()
            self.g_optimizer.step()
            self.q_optimizer.zero_grad()
            self.g_optimizer.zero_grad()
            
            if (it % self.test_iter == 0) or (it == (self.iterations-1)):
                self.qnet.eval()
                self.gnet.eval()
                x_pred = self.gnet(z_train[:800])
                y_pred = self.qnet(x_train[:800], z_train[:800])

                q_loss = self.mse_loss(y_pred, y_train[:800])
                g_loss = self.bce_loss(x_pred, x_train[:800]).mean()
                train_losses_q.append(q_loss.item())
                train_losses_g.append(g_loss.item())
                
                q_loss_test, g_loss_test, _, _ = self.test(x_val, y_val, z_val)
                test_losses_q.append(q_loss_test.item())
                test_losses_g.append(g_loss_test.item())
                print('== Iteration {} =='.format(it))
                print('Test Loss Q:', q_loss_test.item(), '  Test Loss G:', g_loss_test.item())
                
                self.qnet.train()
                self.gnet.train()
        
        return train_losses_q, train_losses_g, test_losses_q, test_losses_g
    
    
    def test(self, x, y, z):
        self.qnet.eval()
        self.gnet.eval()
        
        x_pred = self.gnet(z)
        y_pred = self.qnet(x,z)

        q_loss = self.mse_loss(y_pred, y)
        g_loss = self.bce_loss(x_pred, x).mean()
        
        
        return q_loss, g_loss, x_pred, y_pred
    

## 4. Create a hyperparameter tuning class

In [110]:
class Tuner(object):
    def __init__(self, x, y, z, trials, best_params=None):
        self.best_params = best_params
        self.x = x
        self.y = y
        self.z = z
        self.trials = trials
        self.test_iter = 500
        self.best_params = best_params
        self.qnet = None
        self.gnet = None
        self.best_model_q = None
        self.best_model_g = None
        
    def tune(self):
        
        batch_sizes = [30, 60, 90, 120]
        iterss = [5000, 10000, 15000, 20000, 50000, 100000]
        lrs = [0.0001, 0.0005, 0.001, 0.005, 0.01]
        layerss = [2, 4, 6]
        layer_sizes = [16, 32, 64, 128]
        dropouts = [0.1, 0.2, 0.4]
        output_type_Q = 'categorical'
        output_size_Q = 1
        output_type_G = 'categorical'
        output_size_G = 1
        input_size_Q = z.shape[-1] + 1  # we will concatenate the treatment var inside the qnet class
        input_size_G = z.shape[-1]

        train_loss_q = []
        train_loss_g = []
        val_loss_q = []
        val_loss_g = []
        bs_ = []
        iters_ = []
        lr_ = []
        layers_ = []
        dropout_ = []
        layer_size_ = []
        best_loss = 1e10
        for trial in range(self.trials):
            # sample hyper params and store the history
            bs = np.random.choice(batch_sizes) if self.best_params == None else self.best_params['batch_size']
            bs_.append(bs)
            iters = np.random.choice(iterss) if self.best_params == None else self.best_params['iters']
            iters_.append(iters)
            lr = np.random.choice(lrs) if self.best_params == None else self.best_params['lr']
            lr_.append(lr)
            layers = np.random.choice(layerss) if self.best_params == None else self.best_params['layers']
            layers_.append(layers)
            dropout = np.random.choice(dropouts) if self.best_params == None else self.best_params['dropout']
            dropout_.append(dropout)
            layer_size = np.random.choice(layer_sizes) if self.best_params == None else self.best_params['layer_size']
            layer_size_.append(layer_size)
            print('======== Trial {} of {} ========='.format(trial, self.trials-1))
            print('Batch size', bs, ' Iters', iters, ' Lr', lr, ' Layers', layers,
                 ' Dropout', dropout, ' Layer Size', layer_size)

            

            self.qnet = QNet(input_size=input_size_Q, num_layers=layers,
                      layers_size=layer_size, output_size=output_size_Q,
                     output_type=output_type_Q, dropout=dropout)
        
            self.gnet = GNet(input_size=input_size_G, num_layers=layers,
                      layers_size=layer_size, output_size=output_size_G,
                     output_type=output_type_G, dropout=dropout)


            trainer = Trainer(qnet=self.qnet, gnet=self.gnet, iterations=iters,
                          batch_size=bs, test_iter=self.test_iter, lr=lr)
            train_loss_q_, train_loss_g_, val_loss_q_, val_loss_g_ = trainer.train(self.x,
                                                                                  self.y,
                                                                                  self.z)
            train_loss_q.append(train_loss_q_[-1])
            train_loss_g.append(train_loss_g_[-1])
            val_loss_q.append(val_loss_q_[-1])
            val_loss_g.append(val_loss_g_[-1])
            
            total_val_loss = val_loss_q_[-1] + val_loss_g_[-1]
            
            if total_val_loss < best_loss:
                print('old loss:', best_loss)
                print('new loss:', total_val_loss)
                print('best model updated')
                best_loss = total_val_loss
                self.best_model_q = self.qnet
                self.best_model_g = self.gnet

        tuning_dict = {'batch_size': bs_, 'layers':layers_, 'dropout':dropout_,
                      'layer_size':layer_size_,'lr':lr_, 'iters':iters_,
                      'train_loss_q':train_loss_q, 'train_loss_g':train_loss_g,
                      'val_loss_q':val_loss_q, 'val_loss_g':val_loss_g}
        
        return tuning_dict, self.best_model_q, self.best_model_g
        

## 5. Run Hyperparameter Search

Now we have everything we need, we can initialize the neural networks, run hyperparameter search to identify the best parameters.

In [88]:
# First establish ground truth treatment effect:
N = 5000000
Z, x, y, Y1, Y0 = generate_data(N, seed=0)
true_psi = (Y1-Y0).mean()


# Set some params
N = 10000
seed = 0
num_tuning_trials = 60

# data generation:
z, x, y, _, _ = generate_data(N, 0)
x = torch.tensor(x).type(torch.float32)
z = torch.tensor(z).type(torch.float32)
y = torch.tensor(y).type(torch.float32)
    
tuner = Tuner(x=x,y=y,z=z,trials=num_tuning_trials)
tuning_history, best_q, best_g = tuner.tune()

total_losses = np.asarray(tuning_history['val_loss_g']) + np.asarray(tuning_history['val_loss_q'])
best_index = np.argmin(total_losses)

best_params = {}
for key in tuning_history.keys():
    best_params[key] = tuning_history[key][best_index]

======== Trial 0 of 59 =========
Batch size 90  Iters 5000  Lr 0.0001  Layers 4  Dropout 0.4  Layer Size 128
== Iteration 0 ==
Test Loss Q: 0.23505303263664246   Test Loss G: 0.6583337187767029
== Iteration 500 ==
Test Loss Q: 0.18650640547275543   Test Loss G: 0.6955683827400208
== Iteration 1000 ==
Test Loss Q: 0.18443691730499268   Test Loss G: 0.7021484375
== Iteration 1500 ==
Test Loss Q: 0.1827680766582489   Test Loss G: 0.7017059326171875
== Iteration 2000 ==
Test Loss Q: 0.1817612498998642   Test Loss G: 0.6955196261405945
== Iteration 2500 ==
Test Loss Q: 0.18313805758953094   Test Loss G: 0.6959470510482788
== Iteration 3000 ==
Test Loss Q: 0.1808604896068573   Test Loss G: 0.7102521657943726
== Iteration 3500 ==
Test Loss Q: 0.1827697455883026   Test Loss G: 0.6995244026184082
== Iteration 4000 ==
Test Loss Q: 0.18118160963058472   Test Loss G: 0.7010411620140076
== Iteration 4500 ==
Test Loss Q: 0.18189026415348053   Test Loss G: 0.6989416480064392
== Iteration 4999 ==
Test

== Iteration 6000 ==
Test Loss Q: 0.17381247878074646   Test Loss G: 0.6827932000160217
== Iteration 6500 ==
Test Loss Q: 0.17513717710971832   Test Loss G: 0.6874383091926575
== Iteration 7000 ==
Test Loss Q: 0.1752253919839859   Test Loss G: 0.6938260793685913
== Iteration 7500 ==
Test Loss Q: 0.1740250438451767   Test Loss G: 0.7062925696372986
== Iteration 8000 ==
Test Loss Q: 0.17190100252628326   Test Loss G: 0.7072523236274719
== Iteration 8500 ==
Test Loss Q: 0.1739846020936966   Test Loss G: 0.7038698792457581
== Iteration 9000 ==
Test Loss Q: 0.17233605682849884   Test Loss G: 0.671136736869812
== Iteration 9500 ==
Test Loss Q: 0.17022594809532166   Test Loss G: 0.6880619525909424
== Iteration 10000 ==
Test Loss Q: 0.1711665689945221   Test Loss G: 0.6859276294708252
== Iteration 10500 ==
Test Loss Q: 0.17470912635326385   Test Loss G: 0.7332396507263184
== Iteration 11000 ==
Test Loss Q: 0.17240367829799652   Test Loss G: 0.7153126001358032
== Iteration 11500 ==
Test Loss Q:

== Iteration 20500 ==
Test Loss Q: 0.18782973289489746   Test Loss G: 0.6870893836021423
== Iteration 21000 ==
Test Loss Q: 0.1838790476322174   Test Loss G: 0.6716381311416626
== Iteration 21500 ==
Test Loss Q: 0.18567998707294464   Test Loss G: 0.6854738593101501
== Iteration 22000 ==
Test Loss Q: 0.1870380938053131   Test Loss G: 0.697191059589386
== Iteration 22500 ==
Test Loss Q: 0.18699055910110474   Test Loss G: 0.7257840037345886
== Iteration 23000 ==
Test Loss Q: 0.18581131100654602   Test Loss G: 0.6819648146629333
== Iteration 23500 ==
Test Loss Q: 0.18724779784679413   Test Loss G: 0.6951056718826294
== Iteration 24000 ==
Test Loss Q: 0.18666759133338928   Test Loss G: 0.6761404871940613
== Iteration 24500 ==
Test Loss Q: 0.1819620430469513   Test Loss G: 0.7098820805549622
== Iteration 25000 ==
Test Loss Q: 0.18762798607349396   Test Loss G: 0.7225524187088013
== Iteration 25500 ==
Test Loss Q: 0.18845969438552856   Test Loss G: 0.6848759055137634
== Iteration 26000 ==
Tes

== Iteration 67000 ==
Test Loss Q: 0.1912180483341217   Test Loss G: 0.7923637628555298
== Iteration 67500 ==
Test Loss Q: 0.1908757984638214   Test Loss G: 0.801216185092926
== Iteration 68000 ==
Test Loss Q: 0.19080978631973267   Test Loss G: 0.8068796396255493
== Iteration 68500 ==
Test Loss Q: 0.19428478181362152   Test Loss G: 0.7715816497802734
== Iteration 69000 ==
Test Loss Q: 0.19443123042583466   Test Loss G: 0.8131933808326721
== Iteration 69500 ==
Test Loss Q: 0.1924627125263214   Test Loss G: 0.7988831400871277
== Iteration 70000 ==
Test Loss Q: 0.1962345540523529   Test Loss G: 0.7927321195602417
== Iteration 70500 ==
Test Loss Q: 0.19536404311656952   Test Loss G: 0.7764102816581726
== Iteration 71000 ==
Test Loss Q: 0.19648206233978271   Test Loss G: 0.7911187410354614
== Iteration 71500 ==
Test Loss Q: 0.19154632091522217   Test Loss G: 0.7939262986183167
== Iteration 72000 ==
Test Loss Q: 0.19311034679412842   Test Loss G: 0.8042961359024048
== Iteration 72500 ==
Test

== Iteration 12500 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6982542872428894
== Iteration 13000 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6804891228675842
== Iteration 13500 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6640640497207642
== Iteration 14000 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.67203289270401
== Iteration 14500 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6649522185325623
== Iteration 15000 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6765958070755005
== Iteration 15500 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6912574768066406
== Iteration 16000 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6828961968421936
== Iteration 16500 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.7426938414573669
== Iteration 17000 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6755930185317993
== Iteration 17500 ==
Test Loss Q: 0.3240000009536743   Test Loss G: 0.6835403442382812
== Iteration 18000 ==
Test Loss Q:

...

== Iteration 70000 ==
Test Loss Q: 0.1800711303949356   Test Loss G: 0.6963858008384705
== Iteration 70500 ==
Test Loss Q: 0.17970237135887146   Test Loss G: 0.67628014087677
== Iteration 71000 ==
Test Loss Q: 0.1813019961118698   Test Loss G: 0.6767477989196777
== Iteration 71500 ==
Test Loss Q: 0.1804761290550232   Test Loss G: 0.7042042016983032
== Iteration 72000 ==
Test Loss Q: 0.18000872433185577   Test Loss G: 0.7274227142333984
== Iteration 72500 ==
Test Loss Q: 0.18139415979385376   Test Loss G: 0.7684697508811951
== Iteration 73000 ==
Test Loss Q: 0.18082788586616516   Test Loss G: 0.7876018285751343
== Iteration 73500 ==
Test Loss Q: 0.1802658885717392   Test Loss G: 0.7996652126312256
== Iteration 74000 ==
Test Loss Q: 0.18023991584777832   Test Loss G: 0.7616757154464722
== Iteration 74500 ==
Test Loss Q: 0.18057823181152344   Test Loss G: 0.865592896938324
== Iteration 75000 ==
Test Loss Q: 0.17971543967723846   Test Loss G: 0.7795681357383728
== Iteration 75500 ==
Test L

== Iteration 15500 ==
Test Loss Q: 0.18209704756736755   Test Loss G: 0.6887077689170837
== Iteration 16000 ==
Test Loss Q: 0.18766731023788452   Test Loss G: 0.6888712048530579
== Iteration 16500 ==
Test Loss Q: 0.18419763445854187   Test Loss G: 0.7199059724807739
== Iteration 17000 ==
Test Loss Q: 0.18489857017993927   Test Loss G: 0.6984457969665527
== Iteration 17500 ==
Test Loss Q: 0.18586888909339905   Test Loss G: 0.7129437923431396
== Iteration 18000 ==
Test Loss Q: 0.1831354796886444   Test Loss G: 0.7070049047470093
== Iteration 18500 ==
Test Loss Q: 0.18509498238563538   Test Loss G: 0.7203938961029053
== Iteration 19000 ==
Test Loss Q: 0.1848967969417572   Test Loss G: 0.7015427350997925
== Iteration 19500 ==
Test Loss Q: 0.1859242022037506   Test Loss G: 0.721469521522522
== Iteration 20000 ==
Test Loss Q: 0.1887514889240265   Test Loss G: 0.7357023358345032
== Iteration 20500 ==
Test Loss Q: 0.18457303941249847   Test Loss G: 0.6865736842155457
== Iteration 21000 ==
Test

== Iteration 62000 ==
Test Loss Q: 0.18820138275623322   Test Loss G: 0.7322945594787598
== Iteration 62500 ==
Test Loss Q: 0.1890404224395752   Test Loss G: 0.7717601656913757
== Iteration 63000 ==
Test Loss Q: 0.18546609580516815   Test Loss G: 0.7637732028961182
== Iteration 63500 ==
Test Loss Q: 0.18345347046852112   Test Loss G: 0.7623404860496521
== Iteration 64000 ==
Test Loss Q: 0.1877463459968567   Test Loss G: 0.770463764667511
== Iteration 64500 ==
Test Loss Q: 0.19091515243053436   Test Loss G: 0.7670603394508362
== Iteration 65000 ==
Test Loss Q: 0.1887199878692627   Test Loss G: 0.7545042037963867
== Iteration 65500 ==
Test Loss Q: 0.18981485068798065   Test Loss G: 0.7445299029350281
== Iteration 66000 ==
Test Loss Q: 0.18396499752998352   Test Loss G: 0.7933180928230286
== Iteration 66500 ==
Test Loss Q: 0.18573172390460968   Test Loss G: 0.7837985157966614
== Iteration 67000 ==
Test Loss Q: 0.185205340385437   Test Loss G: 0.762081503868103
== Iteration 67500 ==
Test L

== Iteration 7500 ==
Test Loss Q: 0.17347846925258636   Test Loss G: 0.65046226978302
== Iteration 8000 ==
Test Loss Q: 0.1732139140367508   Test Loss G: 0.6759098768234253
== Iteration 8500 ==
Test Loss Q: 0.1739744395017624   Test Loss G: 0.6761988997459412
== Iteration 9000 ==
Test Loss Q: 0.17386981844902039   Test Loss G: 0.6622137427330017
== Iteration 9500 ==
Test Loss Q: 0.1771010309457779   Test Loss G: 0.6939374804496765
== Iteration 10000 ==
Test Loss Q: 0.1753312200307846   Test Loss G: 0.6760265827178955
== Iteration 10500 ==
Test Loss Q: 0.17532868683338165   Test Loss G: 0.6944417953491211
== Iteration 11000 ==
Test Loss Q: 0.17610780894756317   Test Loss G: 0.6863893866539001
== Iteration 11500 ==
Test Loss Q: 0.17442277073860168   Test Loss G: 0.7205119132995605
== Iteration 12000 ==
Test Loss Q: 0.17487433552742004   Test Loss G: 0.6874871850013733
== Iteration 12500 ==
Test Loss Q: 0.17625857889652252   Test Loss G: 0.6887350678443909
== Iteration 13000 ==
Test Loss 

== Iteration 17000 ==
Test Loss Q: 0.17895053327083588   Test Loss G: 0.672584593296051
== Iteration 17500 ==
Test Loss Q: 0.17914071679115295   Test Loss G: 0.6539864540100098
== Iteration 18000 ==
Test Loss Q: 0.179704487323761   Test Loss G: 0.6493998765945435
== Iteration 18500 ==
Test Loss Q: 0.1785597950220108   Test Loss G: 0.652703583240509
== Iteration 19000 ==
Test Loss Q: 0.17968711256980896   Test Loss G: 0.6489287614822388
== Iteration 19500 ==
Test Loss Q: 0.1787949651479721   Test Loss G: 0.6723790764808655
== Iteration 19999 ==
Test Loss Q: 0.1795770525932312   Test Loss G: 0.6464273929595947
old loss: 0.8367812931537628
new loss: 0.8260044455528259
best model updated


## 6. Run Simulation

Now we have the best hyperparameters, we will run the simulations accordingly

In [151]:
print(best_params)
N = 10000
seed = 0
num_runs = 60

output_type_Q = 'categorical'
output_size_Q = 1
output_type_G = 'categorical'
output_size_G = 1
input_size_Q = z.shape[-1] + 1  # we will concatenate the treatment var inside the qnet class
input_size_G = z.shape[-1]
layers = best_params['layers']
dropout = best_params['dropout']
layer_size = best_params['layer_size']
iters = best_params['iters']
lr = best_params['lr']
batch_size = best_params['batch_size']

estimates_naive = []
estimates_upd = []
for i in range(num_runs):
    print('=====================RUN {}==================='.format(i))
    seed += 1
    # data generation:
    z, x, y, _, _ = generate_data(N, seed=seed)
    x = torch.tensor(x).type(torch.float32)
    z = torch.tensor(z).type(torch.float32)
    y = torch.tensor(y).type(torch.float32)
    x_int1 = torch.ones_like(x)  # this is the 'intervention data'
    x_int0 = torch.zeros_like(x)    

    qnet = QNet(input_size=input_size_Q, num_layers=layers,
                          layers_size=layer_size, output_size=output_size_Q,
                         output_type=output_type_Q, dropout=dropout)

    gnet = GNet(input_size=input_size_G, num_layers=layers,
                          layers_size=layer_size, output_size=output_size_G,
                         output_type=output_type_G, dropout=dropout)


    trainer = Trainer(qnet=qnet, gnet=gnet, iterations=iters,
                      batch_size=batch_size, test_iter=500, lr=lr)

    train_loss_q_, train_loss_g_, val_loss_q_, val_loss_g_ = trainer.train(x, y, z)

    _, _, x_pred, y_pred = trainer.test(x, y, z)
    x_pred, y_pred = x_pred.detach().numpy(), y_pred.detach().numpy()

    _, _, G10, Q1 = trainer.test(x_int1, y, z)
    _, _, _, Q0 = trainer.test(x_int0, y, z)

    Q1 = Q1.detach().numpy()
    Q0 = Q0.detach().numpy()
    biased_psi = (Q1-Q0).mean()

    G10 = np.clip(G10.detach().numpy(), a_min=0.001, a_max=0.999)

    H1 = 1/(G10)
    H0 = 1 / (1 - G10)

    x_ = x.detach().numpy()
    y_ = y.detach().numpy()
    D1 = x_ * H1 * (y_ - Q1) + Q1 - Q1.mean()
    D0 = (1 - x_) * H0 * (y_ - Q0) + Q0 - Q0.mean()

    Q1_star = Q1 + D1
    Q0_star = Q0 + D0

    upd_psi = (Q1_star - Q0_star).mean()

    estimates_naive.append(biased_psi)
    estimates_upd.append(upd_psi)
 


{'batch_size': 60, 'layers': 4, 'dropout': 0.2, 'layer_size': 64, 'lr': 0.005, 'iters': 20000, 'train_loss_q': 0.16899374127388, 'train_loss_g': 0.6496564745903015, 'val_loss_q': 0.1795770525932312, 'val_loss_g': 0.6464273929595947}
=====================RUN 0===================
== Iteration 0 ==
Test Loss Q: 0.20961526036262512   Test Loss G: 0.6375666856765747
== Iteration 500 ==
Test Loss Q: 0.17310187220573425   Test Loss G: 0.704255998134613
== Iteration 1000 ==
Test Loss Q: 0.17446810007095337   Test Loss G: 0.7013257145881653
== Iteration 1500 ==
Test Loss Q: 0.17449995875358582   Test Loss G: 0.7027065753936768
== Iteration 2000 ==
Test Loss Q: 0.17378754913806915   Test Loss G: 0.714412271976471
== Iteration 2500 ==
Test Loss Q: 0.1746959090232849   Test Loss G: 0.6836843490600586
== Iteration 3000 ==
Test Loss Q: 0.17220377922058105   Test Loss G: 0.7178129553794861
== Iteration 3500 ==
Test Loss Q: 0.17506831884384155   Test Loss G: 0.6949360966682434
== Iteration 4000 ==
Tes

== Iteration 4000 ==
Test Loss Q: 0.176979660987854   Test Loss G: 0.6616436243057251
== Iteration 4500 ==
Test Loss Q: 0.17912252247333527   Test Loss G: 0.6680048704147339
== Iteration 5000 ==
Test Loss Q: 0.17734304070472717   Test Loss G: 0.65729159116745
== Iteration 5500 ==
Test Loss Q: 0.17780260741710663   Test Loss G: 0.654613196849823
== Iteration 6000 ==
Test Loss Q: 0.17895393073558807   Test Loss G: 0.642874002456665
== Iteration 6500 ==
Test Loss Q: 0.17671532928943634   Test Loss G: 0.6663792133331299
== Iteration 7000 ==
Test Loss Q: 0.17928814888000488   Test Loss G: 0.6749085187911987
== Iteration 7500 ==
Test Loss Q: 0.17874492704868317   Test Loss G: 0.6385259032249451
== Iteration 8000 ==
Test Loss Q: 0.17718827724456787   Test Loss G: 0.6853234171867371
== Iteration 8500 ==
Test Loss Q: 0.17760993540287018   Test Loss G: 0.6710301041603088
== Iteration 9000 ==
Test Loss Q: 0.177336648106575   Test Loss G: 0.6828089952468872
== Iteration 9500 ==
Test Loss Q: 0.1771

== Iteration 9500 ==
Test Loss Q: 0.18178677558898926   Test Loss G: 0.6556441187858582
== Iteration 10000 ==
Test Loss Q: 0.18131624162197113   Test Loss G: 0.667153000831604
== Iteration 10500 ==
Test Loss Q: 0.1825079321861267   Test Loss G: 0.6684373021125793
== Iteration 11000 ==
Test Loss Q: 0.18446648120880127   Test Loss G: 0.653491735458374
== Iteration 11500 ==
Test Loss Q: 0.18187089264392853   Test Loss G: 0.6582018733024597
== Iteration 12000 ==
Test Loss Q: 0.18322548270225525   Test Loss G: 0.6912370324134827
== Iteration 12500 ==
Test Loss Q: 0.1820419728755951   Test Loss G: 0.6726122498512268
== Iteration 13000 ==
Test Loss Q: 0.18126635253429413   Test Loss G: 0.6588608622550964
== Iteration 13500 ==
Test Loss Q: 0.18243034183979034   Test Loss G: 0.7135289311408997
== Iteration 14000 ==
Test Loss Q: 0.18231719732284546   Test Loss G: 0.657837986946106
== Iteration 14500 ==
Test Loss Q: 0.18447941541671753   Test Loss G: 0.6654677391052246
== Iteration 15000 ==
Test 

== Iteration 15000 ==
Test Loss Q: 0.18620625138282776   Test Loss G: 0.7274943590164185
== Iteration 15500 ==
Test Loss Q: 0.1848737597465515   Test Loss G: 0.6832261681556702
== Iteration 16000 ==
Test Loss Q: 0.18537801504135132   Test Loss G: 0.717913806438446
== Iteration 16500 ==
Test Loss Q: 0.18502943217754364   Test Loss G: 0.689757227897644
== Iteration 17000 ==
Test Loss Q: 0.1853286325931549   Test Loss G: 0.7070826292037964
== Iteration 17500 ==
Test Loss Q: 0.18616870045661926   Test Loss G: 0.7193867564201355
== Iteration 18000 ==
Test Loss Q: 0.1862832009792328   Test Loss G: 0.6815193295478821
== Iteration 18500 ==
Test Loss Q: 0.18526101112365723   Test Loss G: 0.7080547213554382
== Iteration 19000 ==
Test Loss Q: 0.18587148189544678   Test Loss G: 0.696033775806427
== Iteration 19500 ==
Test Loss Q: 0.18674863874912262   Test Loss G: 0.6955092549324036
== Iteration 19999 ==
Test Loss Q: 0.18545366823673248   Test Loss G: 0.6724504232406616
=====================RUN 7=

== Iteration 500 ==
Test Loss Q: 0.19295740127563477   Test Loss G: 0.7065145969390869
== Iteration 1000 ==
Test Loss Q: 0.18333519995212555   Test Loss G: 0.6937850713729858
== Iteration 1500 ==
Test Loss Q: 0.1839827597141266   Test Loss G: 0.72364741563797
== Iteration 2000 ==
Test Loss Q: 0.1832769215106964   Test Loss G: 0.6575930118560791
== Iteration 2500 ==
Test Loss Q: 0.18395625054836273   Test Loss G: 0.6631726026535034
== Iteration 3000 ==
Test Loss Q: 0.18346118927001953   Test Loss G: 0.6960019469261169
== Iteration 3500 ==
Test Loss Q: 0.18285730481147766   Test Loss G: 0.6739802360534668
== Iteration 4000 ==
Test Loss Q: 0.18153272569179535   Test Loss G: 0.6734603047370911
== Iteration 4500 ==
Test Loss Q: 0.1834358274936676   Test Loss G: 0.7261164784431458
== Iteration 5000 ==
Test Loss Q: 0.18321894109249115   Test Loss G: 0.6857438087463379
== Iteration 5500 ==
Test Loss Q: 0.18348002433776855   Test Loss G: 0.76627117395401
== Iteration 6000 ==
Test Loss Q: 0.1836

...

== Iteration 14000 ==
Test Loss Q: 0.17976313829421997   Test Loss G: 0.6572891473770142
== Iteration 14500 ==
Test Loss Q: 0.17982560396194458   Test Loss G: 0.676898717880249
== Iteration 15000 ==
Test Loss Q: 0.18033039569854736   Test Loss G: 0.667427122592926
== Iteration 15500 ==
Test Loss Q: 0.1787419617176056   Test Loss G: 0.6660481095314026
== Iteration 16000 ==
Test Loss Q: 0.17849688231945038   Test Loss G: 0.696666419506073
== Iteration 16500 ==
Test Loss Q: 0.18008731305599213   Test Loss G: 0.6762767434120178
== Iteration 17000 ==
Test Loss Q: 0.18058675527572632   Test Loss G: 0.6814953684806824
== Iteration 17500 ==
Test Loss Q: 0.1800428032875061   Test Loss G: 0.6663069128990173
== Iteration 18000 ==
Test Loss Q: 0.17974835634231567   Test Loss G: 0.6661312580108643
== Iteration 18500 ==
Test Loss Q: 0.1811143308877945   Test Loss G: 0.6887983679771423
== Iteration 19000 ==
Test Loss Q: 0.18103919923305511   Test Loss G: 0.668516993522644
== Iteration 19500 ==
Test L

== Iteration 19000 ==
Test Loss Q: 0.1767008900642395   Test Loss G: 0.6773212552070618
== Iteration 19500 ==
Test Loss Q: 0.17618483304977417   Test Loss G: 0.6686252355575562
== Iteration 19999 ==
Test Loss Q: 0.1757931262254715   Test Loss G: 0.6612309813499451
=====================RUN 52===================
== Iteration 0 ==
Test Loss Q: 0.2119615226984024   Test Loss G: 0.7353114485740662
== Iteration 500 ==
Test Loss Q: 0.19337384402751923   Test Loss G: 0.6783080101013184
== Iteration 1000 ==
Test Loss Q: 0.19336192309856415   Test Loss G: 0.7159327268600464
== Iteration 1500 ==
Test Loss Q: 0.19258245825767517   Test Loss G: 0.7066288590431213
== Iteration 2000 ==
Test Loss Q: 0.1904962956905365   Test Loss G: 0.6842225193977356
== Iteration 2500 ==
Test Loss Q: 0.1936902105808258   Test Loss G: 0.695909857749939
== Iteration 3000 ==
Test Loss Q: 0.19440260529518127   Test Loss G: 0.6781843304634094
== Iteration 3500 ==
Test Loss Q: 0.1915491223335266   Test Loss G: 0.6799047589

== Iteration 3500 ==
Test Loss Q: 0.18478943407535553   Test Loss G: 0.7237517237663269
== Iteration 4000 ==
Test Loss Q: 0.18377278745174408   Test Loss G: 0.687369167804718
== Iteration 4500 ==
Test Loss Q: 0.18267473578453064   Test Loss G: 0.693062961101532
== Iteration 5000 ==
Test Loss Q: 0.18384170532226562   Test Loss G: 0.6915326714515686
== Iteration 5500 ==
Test Loss Q: 0.18339982628822327   Test Loss G: 0.7080062627792358
== Iteration 6000 ==
Test Loss Q: 0.1832306832075119   Test Loss G: 0.7126919031143188
== Iteration 6500 ==
Test Loss Q: 0.18256688117980957   Test Loss G: 0.6895517706871033
== Iteration 7000 ==
Test Loss Q: 0.18266846239566803   Test Loss G: 0.7031660079956055
== Iteration 7500 ==
Test Loss Q: 0.18301111459732056   Test Loss G: 0.659832775592804
== Iteration 8000 ==
Test Loss Q: 0.1832013726234436   Test Loss G: 0.737763524055481
== Iteration 8500 ==
Test Loss Q: 0.18285924196243286   Test Loss G: 0.6800064444541931
== Iteration 9000 ==
Test Loss Q: 0.18

== Iteration 9000 ==
Test Loss Q: 0.18032705783843994   Test Loss G: 0.6748831272125244
== Iteration 9500 ==
Test Loss Q: 0.17963488399982452   Test Loss G: 0.6674371361732483
== Iteration 10000 ==
Test Loss Q: 0.1817970871925354   Test Loss G: 0.6820816397666931
== Iteration 10500 ==
Test Loss Q: 0.1816866248846054   Test Loss G: 0.6751052737236023
== Iteration 11000 ==
Test Loss Q: 0.1782212108373642   Test Loss G: 0.6874918937683105
== Iteration 11500 ==
Test Loss Q: 0.17951752245426178   Test Loss G: 0.6949000954627991
== Iteration 12000 ==
Test Loss Q: 0.17859698832035065   Test Loss G: 0.6689848899841309
== Iteration 12500 ==
Test Loss Q: 0.18097886443138123   Test Loss G: 0.6891542673110962
== Iteration 13000 ==
Test Loss Q: 0.18370085954666138   Test Loss G: 0.6796928644180298
== Iteration 13500 ==
Test Loss Q: 0.18167760968208313   Test Loss G: 0.6669120192527771
== Iteration 14000 ==
Test Loss Q: 0.17890724539756775   Test Loss G: 0.6944514513015747
== Iteration 14500 ==
Test

== Iteration 14000 ==
Test Loss Q: 0.18438486754894257   Test Loss G: 0.7014032006263733
== Iteration 14500 ==
Test Loss Q: 0.183928981423378   Test Loss G: 0.7117851972579956
== Iteration 15000 ==
Test Loss Q: 0.183023601770401   Test Loss G: 0.6945000290870667
== Iteration 15500 ==
Test Loss Q: 0.1828463077545166   Test Loss G: 0.6939460635185242
== Iteration 16000 ==
Test Loss Q: 0.18411384522914886   Test Loss G: 0.6742557883262634
== Iteration 16500 ==
Test Loss Q: 0.18337029218673706   Test Loss G: 0.701088547706604
== Iteration 17000 ==
Test Loss Q: 0.18477758765220642   Test Loss G: 0.6856302618980408
== Iteration 17500 ==
Test Loss Q: 0.1849672794342041   Test Loss G: 0.6931094527244568
== Iteration 18000 ==
Test Loss Q: 0.18218855559825897   Test Loss G: 0.7041956782341003
== Iteration 18500 ==
Test Loss Q: 0.18432675302028656   Test Loss G: 0.7015549540519714
== Iteration 19000 ==
Test Loss Q: 0.18515963852405548   Test Loss G: 0.7041631937026978
== Iteration 19500 ==
Test L

In [152]:
estimates_upd = np.asarray(estimates_upd)
estimates_naive = np.asarray(estimates_naive)

print('True psi: ', true_psi)
print('naive psi: ', estimates_naive.mean(), ' relative bias:',
      (estimates_naive.mean() - true_psi)/true_psi * 100, '%')
print('updated TMLE psi: ', estimates_upd.mean(), ' relative bias:',
      (estimates_upd.mean() - true_psi)/true_psi * 100, '%')
print('Reduction in bias:', np.abs(estimates_naive.mean() - true_psi)/true_psi * 100 - 
     np.abs(estimates_upd.mean() - true_psi)/true_psi * 100, '%')

True psi:  0.1956508
naive psi:  0.2035916  relative bias: 4.058659642824327 %
updated TMLE psi:  0.20075402  relative bias: 2.6083290422539087 %
Reduction in bias: 1.450330600570418 %
